In [14]:
import os
import shutil
from pathlib import Path
import subprocess
import zipfile

In [ ]:
def extract_zip_here(zip_path):
    # Kiểm tra tệp tồn tại
    if not os.path.exists(zip_path):
        print(f"❌ File không tồn tại: {zip_path}")
        return
    
    extract_to = os.path.dirname(zip_path)
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Hoàn tất: {extract_to}")

extract_zip_here("public test.zip")
extract_zip_here("train_20241023.zip")

✅ Đã giải nén tại chỗ: 


In [4]:
def prepare_data(source_dirs, dest_dir):

    dest_images_dir = Path(dest_dir) / "images"
    dest_labels_dir = Path(dest_dir) / "labels"

    dest_images_dir.mkdir(parents=True, exist_ok=True)
    dest_labels_dir.mkdir(parents=True, exist_ok=True)

    for source_dir in source_dirs:
        source_dir = Path(source_dir)

        for item in source_dir.glob("*"):
            if item.suffix.lower() in (".jpg", ".jpeg", ".png"):
                if item.name.startswith("cam_10_00500"):
                    continue

                destination = dest_images_dir / item.name
                shutil.copy2(item, destination)

                # Tìm file nhãn tương ứng
                label_name = item.stem + ".txt"
                label_path = source_dir / label_name
                if label_path.exists():
                    destination = dest_labels_dir / label_name
                    shutil.copy2(label_path, destination)
            elif item.is_dir():
                prepare_data([item], dest_dir)

In [ ]:
source_dirs = [
    "train_20241023/daytime",
    "train_20241023/nighttime",
]
dest_dir = "data/train"
directory = Path(dest_dir) / "images"
a = "cam_10_00500"
prepare_data(source_dirs, dest_dir)


In [12]:
label_mapping = {
    '4': '0',
    '5': '1',
    '6': '2',
    '7': '3'
}

def relabel_file(file_path, mapping):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    updated_lines = []
    for line in lines:
        parts = line.split()
        original_class_id = parts[0]
        if original_class_id in mapping:
            parts[0] = mapping[original_class_id]

        updated_lines.append(" ".join(parts) + "\n")

    with open(file_path, 'w') as file:
        file.writelines(updated_lines)


In [13]:
folder_path = r"data/train/labels" #Labels data

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        relabel_file(file_path, label_mapping)
